In [44]:
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns

from pymongo.mongo_client import MongoClient
from pymongo.server_api import ServerApi

In [45]:
import os
from dotenv import load_dotenv

load_dotenv()

mongodb_URI = os.getenv("mongodb_URI")

In [46]:
uri = mongodb_URI


In [47]:
client = MongoClient(uri, server_api=ServerApi('1'))

In [48]:
try:
    client.admin.command('ping')
    print("Pinged your deployment. You successfully connected to MongoDB!")
except Exception as e:
    print(e)

Pinged your deployment. You successfully connected to MongoDB!


In [49]:
db = client['test']
purchase_collection = db['purchasings']

In [50]:
cursor = purchase_collection.find({}, {
    "ticker": 1,
    "purchaseValue": 1,
    "quantity": 1,
    "purchaseDate": 1,
    "_id": 0
})

In [51]:
data = list(cursor)

In [52]:
df = pd.DataFrame(data)
df

,ticker,purchaseValue,quantity,purchaseDate
0,BBAS3,21.59,2,2025-06-13
1,BBAS3,21.59,1,2025-06-17
2,BBAS3,21.59,4,2025-06-18


In [53]:
# Calcular o valor total investido por linha
df['totalInvestido'] = df['purchaseValue'] * df['quantity']

# Agrupar por data e somar os valores investidos
df_grouped = df.groupby(['purchaseDate', 'ticker']).agg({'totalInvestido': 'sum'}).reset_index()

fig1 = px.line(
    df_grouped,
    x='purchaseDate',
    y='totalInvestido',
    color='ticker',
    title='Investimentos por dia por ativo',
    markers=True,
    labels={'purchaseDate': 'Data da compra', 'totalInvestido': 'Total investido (R$)', 'ticker': 'Ativo'}
)

fig1.show()

In [54]:
df_grouped

,purchaseDate,ticker,totalInvestido
0,2025-06-13,BBAS3,43.18
1,2025-06-17,BBAS3,21.59
2,2025-06-18,BBAS3,86.36


In [55]:
df_grouped = df.groupby(['purchaseDate', 'ticker']).agg({'quantity': 'sum'}).reset_index()

fig2 = px.line(
    df_grouped,
    x='purchaseDate',
    y='quantity',
    color='ticker',
    title='Quantidade total comprada por dia',
    markers=True,
    labels={'purchaseDate': 'Data', 'quantity': 'Quantidade', 'ticker': 'Ativo'}
)
fig2.show()